In [41]:
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, gc, glob, random, re
# import warnings

In [9]:
proj_dir = '/Users/kaheichoi/git_repo/Akkadian-to-English'
data_dir = os.path.join(proj_dir, 'data')

train_csv = pd.read_csv(os.path.join(data_dir, 'train.csv'))
lexicon = pd.read_csv(os.path.join(data_dir, 'OA_Lexicon_eBL.csv'))


In [14]:
lexicon.info();
print(lexicon.columns)
print(lexicon.shape)
print(lexicon.isnull().sum())
lexicon.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39332 entries, 0 to 39331
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   type       39332 non-null  object
 1   form       39330 non-null  object
 2   norm       39332 non-null  object
 3   lexeme     39332 non-null  object
 4   eBL        39332 non-null  object
 5   I_IV       775 non-null    object
 6   A_D        775 non-null    object
 7   Female(f)  22 non-null     object
 8   Alt_lex    886 non-null    object
dtypes: object(9)
memory usage: 2.7+ MB
Index(['type', 'form', 'norm', 'lexeme', 'eBL', 'I_IV', 'A_D', 'Female(f)',
       'Alt_lex'],
      dtype='object')
(39332, 9)
type             0
form             2
norm             0
lexeme           0
eBL              0
I_IV         38557
A_D          38557
Female(f)    39310
Alt_lex      38446
dtype: int64


,type,form,norm,lexeme,eBL,I_IV,A_D,Female(f),Alt_lex
0,word,áb ša-ra-ni,ab šarrānē,ab šarrānē,https://www.ebl.lmu.de/dictionary?word=ab šarrānē,NaN,NaN,NaN,NaN
1,word,áb ša-ra-nim,ab šarrānem,ab šarrānē,https://www.ebl.lmu.de/dictionary?word=ab šarrānē,NaN,NaN,NaN,NaN
2,word,áb-ša-ra-nim,ab šarrānem,ab šarrānē,https://www.ebl.lmu.de/dictionary?word=ab šarrānē,NaN,NaN,NaN,NaN
3,word,áb-ša-ra-ni,ab šarrānē,ab šarrānē,https://www.ebl.lmu.de/dictionary?word=ab šarrānē,NaN,NaN,NaN,NaN
4,word,áb-ša-ra-ni₇,ab šarrānē,ab šarrānē,https://www.ebl.lmu.de/dictionary?word=ab šarrānē,NaN,NaN,NaN,NaN


In [15]:
train_csv.info();
print(train_csv.columns)
print(train_csv.shape)
print(train_csv.isnull().sum())
train_csv.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1561 entries, 0 to 1560
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   oare_id          1561 non-null   object
 1   transliteration  1561 non-null   object
 2   translation      1561 non-null   object
dtypes: object(3)
memory usage: 36.7+ KB
Index(['oare_id', 'transliteration', 'translation'], dtype='object')
(1561, 3)
oare_id            0
transliteration    0
translation        0
dtype: int64


,oare_id,transliteration,translation
0,004a7dbd-57ce-46f8-9691-409be61c676e,KIŠIB ma-nu-ba-lúm-a-šur DUMU ṣí-lá-(d)IM KIŠI...,"Seal of Mannum-balum-Aššur son of Ṣilli-Adad, ..."
1,0064939c-59b9-4448-a63d-34612af0a1b5,1 TÚG ša qá-tim i-tur₄-DINGIR il₅-qé,Itūr-ilī has received one textile of ordinary ...
2,0073f2c0-524c-4bbf-915a-8c1772a4fb98,TÚG u-la i-dí-na-ku-um i-tù-ra-ma 9 GÍN KÙ.BABBAR,... he did not give you a textile. He returned...
3,009fb838-8038-42bc-ad34-5f795b3840ee,KIŠIB šu-(d)EN.LÍL DUMU šu-ku-bi-im KIŠIB ṣí-l...,"Seal of Šu-Illil son of Šu-Kūbum, seal of Ṣilū..."
4,00aa1c55-c80c-4346-a159-73ad43ab0ff7,um-ma šu-ku-tum-ma a-na IŠTAR-lá-ma-sí ù ni-ta...,From Šukkutum to Ištar-lamassī and Nitahšušar:...


In [29]:
train = {}

train['oare_id'] = train_csv['oare_id']
train['translit_length_char'] = train_csv['transliteration'].str.len()
train['translit_length_word'] = train_csv['transliteration'].str.split().str.len()
train['translat_length_char'] = train_csv['translation'].str.len()
train['translat_length_word'] = train_csv['translation'].str.split().str.len()

display(pd.DataFrame(train))

,oare_id,translit_length_char,translit_length_word,translat_length_char,translat_length_word
0,004a7dbd-57ce-46f8-9691-409be61c676e,374,50,391,70
1,0064939c-59b9-4448-a63d-34612af0a1b5,36,6,54,8
2,0073f2c0-524c-4bbf-915a-8c1772a4fb98,49,7,74,16
3,009fb838-8038-42bc-ad34-5f795b3840ee,186,23,193,36
4,00aa1c55-c80c-4346-a159-73ad43ab0ff7,588,77,553,95
...,...,...,...,...,...
1556,ff3208e4-8ab8-4368-b4df-7b80afa5bc32,597,72,658,122
1557,ff43a284-3d67-4238-8b4a-9b6cb7531e0a,168,21,176,27
1558,ff5747a4-af8a-4100-a906-a2660ae72606,743,88,902,147
1559,ff777871-97ce-4bfc-bdfb-73352868944d,608,75,706,124


In [45]:
from collections import Counter

def find_special_char(text_series):
    all_chars = ''.join(text_series.fillna(''))
    char_counts = Counter(all_chars)

    special_chars = {char: count for char, count in char_counts.items() if not char.isalnum() and not char.isspace()}

    return special_chars

find_special_char(train_csv['transliteration'])

{'-': 119866,
 '(': 1103,
 ')': 1098,
 '.': 8625,
 '…': 1687,
 '[': 210,
 ']': 210,
 '+': 16,
 '⌈': 1}

In [ ]:
train_csv['transliteration'].str()